# S2W7D2: 拥抱 Trainer API (The Industrial Standard)

昨天我们造好了“车”（模型结构 `BertClassifier`），今天我们要给它装上“引擎”（`Trainer`）并注入“燃料”（数据），让它真正跑起来。

在此之前，很多同学（包括以前的我）喜欢手写 `for batch in dataloader:` 循环。但在大厂生产环境中，我们几乎全部转向了 **Hugging Face Trainer API**。

为什么？因为手写循环要自己处理：

  * ❌ 分布式训练 (Multi-GPU)
  * ❌ 梯度累积 (Gradient Accumulation)
  * ❌ 混合精度训练 (FP16)
  * ❌ 日志记录与断点续训

而 Trainer 把这些脏活累活全包了。我们需要掌握这个**工业界标准**。

## 1 🛠️ 核心组件：构建数据集 (Dataset)

Trainer需要吃数据。为了符合我们定义的工业级目录结构，请在 `src/` 下新建 `dataset.py`。我们将把数据处理逻辑封装在这里，保持主流程干净。

**文件路径**: `project_root/src/dataset.py`

```python
import torch
from torch.utils.data import Dataset

class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        """
        :param texts: 文本列表 ["打开灯", "向左转"]
        :param labels: 标签列表 [0, 1]
        :param tokenizer: 分词器实例
        :param max_len: 最大截断长度
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length', # 注意：这里暂时用 max_length，后面会讲动态 padding
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        # Trainer 要求返回字典，且 key 必须能对应上模型 forward 的参数名
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }
```

## 2 💻 笔记本实战：跑通训练流程

### 2.1 环境配置与模拟数据

In [1]:
import sys
sys.path.append("../../")

import torch
from transformers import Trainer, TrainingArguments, BertTokenizer
from src.config import PRETRAINED_MODEL_DIR, CHECKPOINT_DIR, LOG_DIR
from src.models.model_bert import BertClassifier
from src.dataset.dataset import IntentDataset # 假设你刚才把 dataset.py 放在 src/dataset/ 下，或者直接放在 src/ 下

# 1. 模拟一点“机器人指令”数据 (用于跑通流程)
train_texts = [
    "向前走", "向后退", "左转", "右转", "停下", "别动", "抓取杯子", "放下物体",
    "前进十米", "后退五步", "向左看", "向右看", "停止移动", "紧急制动", "开启夹爪", "松开夹爪"
]
# 假设 0:移动, 1:停止, 2:操作
train_labels = [0, 0, 0, 0, 1, 1, 2, 2, 0, 0, 0, 0, 1, 1, 2, 2]

# 2. 初始化 Tokenizer 和 Dataset
print("正在处理数据...")
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_DIR)
# 实例化我们在 src/dataset.py 里写的类
train_dataset = IntentDataset(train_texts, train_labels, tokenizer)

# 检查一下数据长什么样 (Debug 习惯)
print(f"样本示例: {train_dataset[0]}")

正在处理数据...
样本示例: {'input_ids': tensor([ 101, 1403, 1184, 6624,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 

### 2.2 核心配置 (TrainingArguments) —— 重点！

这是面试中会问“你怎么调参”的地方。

In [2]:
training_args = TrainingArguments(
    output_dir=str(CHECKPOINT_DIR),  # 权重保存路径 (src/config.py 定义好的)
    
    # --- 核心超参数 ---
    num_train_epochs=3,              # 跑几轮
    per_device_train_batch_size=2,   # 显存小就设小点 (2, 4, 8, 16)
    learning_rate=2e-5,              # BERT 微调黄金学习率 (1e-5 ~ 5e-5)
    
    # --- 优化策略 ---
    logging_steps=2,                 # 多少步打印一次日志
    save_strategy="epoch",           # 每个 epoch 保存一次权重
    seed=42,                         # 设定随机种子，保证可复现
    
    # --- 硬件设置 ---
    no_cuda=False,                   # 有 GPU 肯定用 GPU
)

print("训练参数配置完成。")

训练参数配置完成。


### 第三步：点火起飞 (Trainer Loop)

注意：因为我们的 `BertClassifier` 输出的是 `logits` 而不是 `loss`（原生 BERT 输出会包含 loss），Trainer 默认需要模型返回 `loss`。

  * **方法 A**: 修改 `model_bert.py`，在 forward 里计算 loss。
  * **方法 B (推荐)**: 自定义 `compute_loss` (稍微高级一点，但我们今天先用简单的方法)。

**修改 `src/models/model_bert.py` 的 Forward 函数 (Day 1 代码补充)：**
为了配合 Trainer，我们需要让模型自己计算 Loss。请回到 `src/models/model_bert.py`，修改 `forward` 部分：

```python
    # 修改 src/models/model_bert.py
    def forward(self, input_ids, attention_mask, labels=None, token_type_ids=None): # 把 token_type_ids 加回来，虽然有时候不用，但为了兼容性最好加上
        outputs = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooler_output = outputs.pooler_output
        pooler_output = self.dropout(pooler_output)
        logits = self.classifier(pooler_output)
        
        loss = None
        if labels is not None:
            # view(-1) 将 labels 展平成一维向量 [batch_size]，防止出现 [batch_size, 1] 的情况
            # 这行代码是修复这个报错的核心！
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))
            
            return (loss, logits)
        
        return logits
```

*(请务必先去修改 `.py` 文件，再运行下面的 Notebook 代码)*

In [3]:
model = BertClassifier(model_path=PRETRAINED_MODEL_DIR, num_labels=3)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer
)

print("🚀 开始训练...")
trainer.train()

/tmp/ipykernel_1358/664258046.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🚀 开始训练...


Step,Training Loss
2,1.302100
4,1.052900
6,1.000500
8,1.011500
10,1.031800
12,0.871700
14,0.769000
16,0.985300
18,0.698800
20,0.970100


TrainOutput(global_step=24, training_loss=0.938076933224996, metrics={'train_runtime': 14.4898, 'train_samples_per_second': 3.313, 'train_steps_per_second': 1.656, 'total_flos': 0.0, 'train_loss': 0.938076933224996, 'epoch': 3.0})

## 3 💥 面试深挖：Dynamic Padding 与 DataCollator

面试官看着你的代码问：*“我在你的Dataset里看到了 `padding='max_length'`，这意味着所有句子都被强制补齐到了 128 长度。如果大部分句子只有 10 个字，这是不是极大的浪费？”*

  * **Static Padding (静态补齐)**:
      * Dataset 里写死 `max_len=128`。
      * 句子 "Stop" (长度1) -\> 补 127 个 0。
      * **缺点**: 显存浪费，计算浪费。
  * **Dynamic Padding (动态补齐)**:
      * Dataset 里**不补齐** (只截断)。
      * 在 `DataLoader` 取出一个 Batch 的时候，看这个 Batch 里最长的句子是多长（比如 15），然后把其他句子补齐到 15。
      * **优点**: 极大地节省显存，训练速度快 30% 以上。

**🔧 优化代码 (加分项)**:
在 `Trainer` 中使用 `DataCollatorWithPadding` 实现动态补齐。

```python
from transformers import DataCollatorWithPadding

# Dataset 里去掉 padding='max_length'
# encoding = tokenizer(..., padding=False, ...)

# Trainer 里加上
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(..., data_collator=data_collator)
```

*(今天先跑通静态 Padding 即可，明天优化时我们再细讲这个)*

## 4\. ⚔️ 每日算法题 (LeetCode 5)

既然是处理序列，今天挑战经典的“回文串”。

### 🎯 题目: [LeetCode 5](../../LeetCode%20practice/1-50.ipynb). 最长回文子串 (Longest Palindromic Substring)

  * **难度**: Medium
  * **描述**: 给你一个字符串 `s`，找到 `s` 中最长的回文子串。
      * 输入: `babad` -\> 输出: `bab` 或 `aba`
  * **思路 (中心扩散法)**:
      * 回文串有什么特点？它是**镜像**的。
      * 遍历字符串每个字符，以此为**中心**，向左右两边扩散，直到两边字符不一样为止。
      * **注意**: 中心可能是一个字符 (aba)，也可能是两个字符 (abba)。

<!-- end list -->

```python
# 伪代码思路
for i in range(len(s)):
    len1 = expand(center=i)      # 奇数长度
    len2 = expand(center=i, i+1) # 偶数长度
    max_len = max(len1, len2)
```


### ✅ Day 2 任务清单

1.  **新建文件**: `src/dataset.py`，实现 `IntentDataset`。
2.  **修改模型**: 更新 `model_bert.py` 的 `forward` 函数，加入 loss 计算逻辑（这是 Trainer 运行的前提）。
3.  **运行 Notebook**: 跑通 `S2W7D2_Trainer_Flow.ipynb`，看到进度条跑完，没有报错。
4.  **算法**: 通过 LeetCode 5。

**特别提醒**: 修改 `.py` 文件后，Notebook 如果已经 import 过，**必须 Restart Kernel** 才能生效！